In [21]:
%load_ext autoreload
%autoreload 2

from perturbations import *
from representation import *
from ls import *
import jupyter_utils as utils

import numpy as np
import matplotlib.pyplot as plt
import tsplib95

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# load problem from the library
problem = tsplib95.load("./archives/bayg29.tsp/bayg29.tsp")
#  ./archives/bayg29.tsp/bayg29.tsp
n = problem.dimension
distance_matrix = utils.weights_to_matrix(problem.edge_weights)
print("size of matrix: ", n)
# best = 1610
fitness = fitness_wrapper(distance_matrix)

size of matrix:  29


In [34]:
perturbation = perturb_move
#           perturb_move
#           perturb_invert
#           perturb_switch

ls = ls_first_improvement

init_fn = better_init_wrapper(size=n, dist_matrix=distance_matrix)

max_evals = 10_000
trials = 50


fitness_list = []


for trial in range(trials):

    results = ls(

        fitness_fn=fitness,
        initialisation_fn=init_fn,
        perturbation_fn=perturbation,
        stop_cond=max_evals,
    )


    fitness_list.append(results["best_fitness"])



fitness_list = np.array(fitness_list)


print(np.mean(fitness_list), np.std(fitness_list))
print(fitness_list)

1884.52 128.2175089447615
[1827. 1881. 1870. 2031. 1894. 1794. 1811. 1839. 1766. 1860. 2008. 1960.
 2036. 2314. 1905. 2081. 1801. 2040. 1842. 1953. 1850. 1950. 1761. 1807.
 1997. 1768. 1677. 1804. 2023. 1817. 1818. 1966. 1735. 1937. 1964. 1864.
 1960. 2114. 1751. 1688. 1797. 1826. 1725. 1805. 1951. 1801. 2101. 1842.
 1984. 1630.]


In [38]:
perturbation = perturb_switch
#           perturb_move
#           perturb_invert
#           perturb_switch

ls = ls_first_improvement_with_reinit

init_fn = better_init_wrapper(size=n, dist_matrix=distance_matrix)

max_evals = 10_000
trials = 50

fitness_list = []
for trial in range(trials):

    results = ls(
        fitness_fn=fitness,
        initialisation_fn=init_fn,
        perturbation_fn=perturbation,
        stop_cond=max_evals,
    )

    fitness_list.append(results["best_fitness"])


fitness_list = np.array(fitness_list)

print(np.mean(fitness_list), np.std(fitness_list))
print(fitness_list)

1991.92 154.93131897715196
[1772. 2106. 1851. 2177. 1892. 1844. 1981. 2108. 1822. 2119. 1764. 1791.
 2079. 1869. 1913. 1949. 1960. 2217. 1985. 2423. 2128. 2039. 1859. 1735.
 1977. 2009. 1878. 1866. 2222. 2012. 1989. 2079. 2049. 1894. 1881. 2144.
 1930. 1739. 1739. 2187. 2084. 2029. 2029. 1962. 2169. 1909. 2210. 2336.
 1939. 1951.]
